<a href="https://colab.research.google.com/github/belfi911/Codewars/blob/main/%D0%9D%D0%BE%D1%80%D0%BC%D0%A2%D0%97(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import time

In [18]:
# FILE = 'Общий_октябрьдля_скрипта_мини_версия.xlsx'

# Названия файлов:
# Лист -> Файл
# отклонение по срока_1 -> Поступления товаров
# прогноз_Бмесяц> -> Прогноз месяца
# отклонение по продажам -> Продажи за 2 года
# за 4 года -> Продажи за 4 года

# «Следить за названиями столбцов» !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

FORMAT = '%d.%m.%Y %H:%M:%S'

start_time = time.time()

data = pd.read_excel(
    f'Поступления товаров.xlsx',
    # sheet_name='отклонение по срока_1',
    skiprows=0,
    usecols='A:F',
    converters={'Код "Инфор"': str,
                'Период': lambda x: pd.to_datetime(x, format=FORMAT),
                'Дата': lambda x: pd.to_datetime(x, format=FORMAT)})
sales_variance = pd.read_excel(
    f'Продажи за 2 года.xlsx',
    # sheet_name='отклонение по продажам',
    skiprows=0,
    usecols='A:Y',
    converters={'Код "Инфор"': str,})
forecast = pd.read_excel(
    f'Прогноз января.xlsx',
    #sheet_name='прогноз_октябрь',
    skiprows=0,
    usecols='A:M',
    converters={'infor': str,})
for_4_years = pd.read_excel(
    f'Продажи за 4 года.xlsx',
    #sheet_name='за 4 года',
    skiprows=0,
    usecols='A:C',
    converters={'Код "Инфор"': str,})

print('...Данные считаны...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))


...Данные считаны...
--- Прошло 64.40779328346252 секунд ---


In [16]:
# ЭТАП 11
forecast['Среднемесячный прогноз'] = forecast.iloc[:, 1:13].mean(axis=1)
forecast['Среднемесячный прогноз'] = forecast['Среднемесячный прогноз'].fillna(0)

print('...ЭТАП 11 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 1----------------- Срок доставки
data['Срок доставки'] = (data['Период'] - data['Дата'])
data['Срок доставки'] = data['Срок доставки'].fillna(pd.Timedelta(0))

print('...ЭТАП 1 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# ЭТАП 5
data['Срок поставки +Срок производства'] = (data['Срок доставки (дней)'] + data['Срок производства (дней)'])
data['Срок поставки +Срок производства'] = data['Срок поставки +Срок производства'].fillna(0)

print('...ЭТАП 5 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 2----------------- Верхний квартиль
data_unique_code = data.loc[:]
data_unique_code = data_unique_code.drop_duplicates('Код "Инфор"')

def high_quantile_by_id(row):
    id = row['Код "Инфор"']
    temp_df = data.loc[data['Код "Инфор"'] == id]
    return temp_df['Срок доставки'].quantile([0.75])

data_unique_code['Верхний квартиль'] = data_unique_code.apply(lambda row: high_quantile_by_id(row), axis=1)
data_unique_code['Верхний квартиль'] = data_unique_code['Верхний квартиль'].fillna(pd.Timedelta(0))

print('...ЭТАП 2 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 3----------------- Поступления из верхнего квартиля
def income_from_quantile(row):
    id = row['Код "Инфор"']
    quantile = data_unique_code.loc[data_unique_code['Код "Инфор"'] == id]
    answer = None
    if (row['Срок доставки'] >= quantile['Верхний квартиль']).bool():
        answer = row['Срок доставки']
    return answer
    
data['Поступления из верхнего квартиля'] = data.apply(lambda row: income_from_quantile(row), axis=1)
data['Поступления из верхнего квартиля'] = data['Поступления из верхнего квартиля'].fillna(pd.Timedelta(0))

print('...ЭТАП 3 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 4----------------- Медиана верхнего квартиля
def median_by_id(row):
    id = row['Код "Инфор"']
    temp_df = data.loc[np.logical_and(
        data['Код "Инфор"'] == id, data['Поступления из верхнего квартиля'] != pd.Timedelta(0))]
    return temp_df['Поступления из верхнего квартиля'].median()

data_unique_code['Медиана верхнего квартиля'] = data_unique_code.apply(lambda row: median_by_id(row), axis=1)
data_unique_code['Медиана верхнего квартиля'] = data_unique_code['Медиана верхнего квартиля'].fillna(pd.Timedelta(0))

print('...ЭТАП 4 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 5----------------- Срок поставки +Срок производства
# Посчитано в этапе 1

# -----------------ЭТАП 6----------------- Отклонение по срокам поставки
data_unique_code['Срок поставки +Срок производства'] = pd.to_timedelta(data_unique_code['Срок поставки +Срок производства'], unit='d')
data_unique_code['Отклонение по срокам поставки'] = np.where(
    data_unique_code['Медиана верхнего квартиля'] > data_unique_code['Срок поставки +Срок производства'],
    (data_unique_code['Медиана верхнего квартиля'] - data_unique_code['Срок поставки +Срок производства']) / data_unique_code['Срок поставки +Срок производства'],
    False
)
data_unique_code['Отклонение по срокам поставки'] = data_unique_code['Отклонение по срокам поставки'].fillna(pd.Timedelta(0))

print('...ЭТАП 6 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 7----------------- Граница верхнего квартиля
def stage7_quantile(row):
    df_stage7 = row.iloc[1:25]
    return df_stage7.quantile([0.75])

sales_variance['Граница верхнего квартиля'] = sales_variance.apply(lambda row: stage7_quantile(row), axis=1)
sales_variance['Граница верхнего квартиля'] = sales_variance['Граница верхнего квартиля'].fillna(0)

print('...ЭТАП 7 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 8----------------- Медиана верхнего квартиля
def stage8_median(row):
    df_stage8 = row.iloc[1:25]
    df_stage8 = df_stage8[df_stage8 >= row['Граница верхнего квартиля']]
    return df_stage8.median()

sales_variance['Медиана верхнего квартиля'] = sales_variance.apply(lambda row: stage8_median(row), axis=1)
sales_variance['Медиана верхнего квартиля'] = sales_variance['Медиана верхнего квартиля'].fillna(0)

print('...ЭТАП 8 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 9----------------- Среднемесячный прогноз
sales_variance = pd.merge(
    sales_variance,
    forecast[['infor', 'Среднемесячный прогноз']],
    how='left',
    left_on='Код "Инфор"',
    right_on='infor')
sales_variance['Среднемесячный прогноз'].fillna(0)


print('...ЭТАП 9 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 10----------------- Отклонение по продажам
sales_variance['Отклонение по продажам'] = np.where(
    sales_variance['Медиана верхнего квартиля'] > sales_variance['Среднемесячный прогноз'],
    (sales_variance['Медиана верхнего квартиля'] - sales_variance['Среднемесячный прогноз']) / sales_variance['Среднемесячный прогноз'],
    False
)

print('...ЭТАП 10 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 11----------------- Среднемесячный прогноз
# Посчитано вначале, после считывания forecast

# -----------------ЭТАП 12----------------- Отклонение по продажам
forecast = pd.merge(
    forecast,
    sales_variance[['Код "Инфор"', 'Отклонение по продажам']],
    how='left',
    left_on='infor',
    right_on='Код "Инфор"')
forecast['Отклонение по продажам'].fillna(0)

print('...ЭТАП 12 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 13----------------- Отклонение по срокам поставки
forecast = pd.merge(
    forecast,
    data_unique_code[['Код "Инфор"', 'Отклонение по срокам поставки']],
    how='left',
    left_on='infor',
    right_on='Код "Инфор"')
forecast['Отклонение по срокам поставки'].fillna(0)

print('...ЭТАП 13 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 14----------------- Итоговое отклонение
forecast['Итоговое отклонение'] = np.where(
    (forecast['Отклонение по продажам'] + forecast['Отклонение по срокам поставки']) > 0.5,
    0.5,
    forecast['Отклонение по продажам'] + forecast['Отклонение по срокам поставки']
)
forecast['Итоговое отклонение'] = forecast['Итоговое отклонение'].fillna(0)

print('...ЭТАП 14 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 15----------------- Срок доставки+срок производства, дни
for_4_years['Срок производства (дней)'] = for_4_years['Срок производства (дней)'].fillna(0)
for_4_years['Срок доставки (дней)'] = for_4_years['Срок доставки (дней)'].fillna(0)
for_4_years['Срок доставки+срок производства, дни'] = for_4_years['Срок производства (дней)'] + for_4_years['Срок доставки (дней)']
forecast = pd.merge(
    forecast,
    for_4_years[['Код "Инфор"', 'Срок доставки+срок производства, дни']],
    how='left',
    left_on='infor',
    right_on='Код "Инфор"')

print('...ЭТАП 15 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 16----------------- Срок доставки+срок производства, мес
forecast['Срок доставки+срок производства, мес'] = forecast['Срок доставки+срок производства, дни'] / 30
forecast['Срок доставки+срок производства, мес'] = forecast['Срок доставки+срок производства, мес'].fillna(0)
print('...ЭТАП 16 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 17----------------- Целое
forecast['Целое'] = np.where(
    forecast['Срок доставки+срок производства, мес'] < 13,
    forecast['Срок доставки+срок производства, мес'].astype(int),
    12
)

print('...ЭТАП 17 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 18----------------- Остаток       # Здесь изменил формулу надо потестить. На мини версии все ок
forecast['Остаток'] = np.where(
    forecast['Срок доставки+срок производства, мес'] < 13,
    forecast['Срок доставки+срок производства, мес'] - forecast['Срок доставки+срок производства, мес'].astype(int),
    forecast['Срок доставки+срок производства, мес'] - forecast['Целое']
)

print('...ЭТАП 18 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 19----------------- Прогноз на время поставки
def forecast_on_time_delivery(row):
    month = row['Целое']
    if month == 0:
        return row[1] * row['Остаток']
    if month == 1:
        return row[1] + (row[2]*row['Остаток'])
    if 1 < month < 13:
        return row[1:(month+1)].sum() + row[month+1]*row['Остаток']
    else:
        return 0

forecast['Прогноз на время поставки'] = forecast.apply(lambda row: forecast_on_time_delivery(row), axis=1)

print('...ЭТАП 19 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------ЭТАП 20----------------- Итоговое значение с учетом отклонений
forecast['Итоговое значение с учетом отклонений'] = (forecast['Прогноз на время поставки'] + (forecast['Прогноз на время поставки'] * forecast['Итоговое отклонение'])).round(0)

print('...ЭТАП 20 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

# -----------------Итог для 1С-----------------
df_for_1C = forecast[['Код "Инфор"', 'Итоговое значение с учетом отклонений']]
df_for_1C = df_for_1C.rename(columns={
    'Код "Инфор"': 'Код_Инфор',
    'Итоговое значение с учетом отклонений': 'Значение'})

# -----------------ВЫВОД-----------------
# data.to_excel("1_data.xlsx")
# data_unique_code.to_excel("2_data_unique_code.xlsx")
# sales_variance.to_excel("3_sales_variance.xlsx")
# forecast.to_excel("4_forecast.xlsx")
# for_4_years.to_excel("5_for_4_years.xlsx")
df_for_1C.to_excel("6_df_for_1C.xlsx")

print('...ВСЕ ГОТОВО...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))

...ЭТАП 11 завершен...
--- Прошло 3038.3376038074493 секунд ---
...ЭТАП 1 завершен...
--- Прошло 3038.344769716263 секунд ---
...ЭТАП 5 завершен...
--- Прошло 3038.349785089493 секунд ---


KeyboardInterrupt: ignored

In [12]:
print(forecast)

         infor  01.01.2023  01.02.2023  01.03.2023  01.04.2023  01.05.2023  \
0      0010778          27          12          33          44          11   
1      0039713         131         173         320         253         285   
2      0061069          27          42          39          40          49   
3      0010656          32          40          45          50          46   
4      0036956          53          75          84          94          85   
...        ...         ...         ...         ...         ...         ...   
12895  0124756           0           2           2           2           2   
12896  0126916           9          12           0          58           0   
12897  0132515           0           0           0           0           0   
12898  0173137           0           0           0           0           0   
12899  0191692           0           0           0           0           0   

       01.06.2023  01.07.2023  01.08.2023  01.09.2023  ...  01.

In [14]:
forecast['Целое'] = np.where(
    forecast['Срок доставки+срок производства, мес'] < 13,
    forecast['Срок доставки+срок производства, мес'].astype(int), 12
)

print('...ЭТАП 17 завершен...')
print("--- Прошло %s секунд ---" % (time.time() - start_time))



IntCastingNaNError: ignored